# Duck iris prediction

In [20]:
%load_ext magic_duckdb

The magic_duckdb extension is already loaded. To reload it, use:
  %reload_ext magic_duckdb


In [21]:
import duckdb

# Open a connection to your DuckDB database
con = duckdb.connect("my-data.duckdb")

In [22]:
%%dql -co con
CREATE TABLE IF NOT EXISTS birds AS SELECT * FROM read_csv('../../data/birds.csv');
CREATE TABLE IF NOT EXISTS ducks AS SELECT * FROM read_csv('../../data/ducks.csv');

,Count


In [82]:
%%dql -co con
-- create raw training data
CREATE OR REPLACE TABLE prepared_data AS
SELECT
    Species_Common_Name as species,
    Beak_Width AS beak_width,
    Beak_Length_Culmen AS beak_length,
    Wing_Length AS wing_length,
    Tail_Length AS tail_length
FROM birds
INNER JOIN ducks ON name = Species_Common_Name
WHERE (beak_width is not null and beak_length is not null and wing_length is not null and tail_length is not null)
and (species like 'Harlequin%' or species like 'Egyptian%' or species like 'Pink%' or species like 'King%' or species like 'Muscovy%'
or species like 'Fueg%' or species like 'Smew%' or species like 'Laysan%');

-- create target mapping
CREATE OR REPLACE TABLE target_mapping AS
SELECT species,
       ROW_NUMBER() OVER (ORDER BY species) - 1 AS target
FROM (SELECT DISTINCT species FROM prepared_data);

-- create ml features
CREATE OR REPLACE TABLE ml_data AS
SELECT
    t.target as y,
    [p.beak_width, p.beak_length, p.wing_length, p.tail_length] as X
FROM prepared_data p
JOIN target_mapping t ON p.species = t.species;

,Count
0,48


# SQL Sampling

In [83]:
%%dql -co con
ALTER TABLE ml_data ADD COLUMN IF NOT EXISTS random_value DOUBLE;
UPDATE ml_data SET random_value = random();

,Count
0,48


In [84]:
%%dql -co con
CREATE OR REPLACE TABLE train_data AS
WITH stratified_data AS (
    SELECT *,
           ROW_NUMBER() OVER (PARTITION BY y ORDER BY random_value) AS row_num,
           COUNT(*) OVER (PARTITION BY y) AS class_count
    FROM ml_data
)
SELECT
    y, X
FROM stratified_data
WHERE row_num <= CEIL(0.65 * class_count);

,Count
0,33


In [85]:
%%dql -co con
from prepared_data

,species,beak_width,beak_length,wing_length,tail_length
0,Egyptian goose,22.2,53.0,403.0,140.0
1,Egyptian goose,21.7,45.2,392.0,148.0
2,Egyptian goose,19.1,45.2,360.0,113.0
3,Egyptian goose,20.5,48.0,350.0,112.0
4,Egyptian goose,20.3,62.7,344.0,138.0
5,Egyptian goose,20.7,55.5,287.0,122.0
6,Laysan duck,13.8,43.0,204.0,98.0
7,Laysan duck,14.6,46.0,213.0,72.0
8,Laysan duck,14.4,44.0,198.0,70.0
9,Laysan duck,12.7,41.1,184.0,94.0


In [86]:
%%dql -co con
CREATE OR REPLACE TABLE test_data AS
SELECT
    y, X
FROM ml_data
EXCEPT
SELECT
    y, X
FROM train_data;

,Count
0,15


In [87]:
df = con.execute("""SELECT * FROM train_data ORDER BY y""").fetchnumpy()
X = df['X'].tolist()
y = df['y'].tolist()

In [88]:
# sklearn training
from sklearn import tree
classifier=tree.DecisionTreeClassifier()
classifier.fit(X,y)

# Register sklearn model in Python UDF
def predict_duck(X):
    pred = classifier.predict([X])[0]
    return int(pred)

con.create_function("predict_duck", predict_duck, ['DOUBLE[]'], 'INTEGER')

In [89]:
%%dql -co con
FROM test_data ORDER BY y

,y,X
0,0,"[21.7, 45.2, 392.0, 148.0]"
1,0,"[20.3, 62.7, 344.0, 138.0]"
2,1,"[31.4, 61.4, 264.0, 97.0]"
3,2,"[12.5, 25.9, 196.0, 96.0]"
4,2,"[15.4, 31.4, 203.0, 103.0]"
5,3,"[16.9, 34.8, 272.0, 80.0]"
6,4,"[13.8, 43.0, 204.0, 98.0]"
7,4,"[14.2, 47.5, 207.0, 93.0]"
8,5,"[23.9, 71.1, 323.0, 190.0]"
9,5,"[20.5, 59.1, 272.0, 151.0]"


In [90]:
%%dql -co con
-- make predictions on test data
CREATE OR REPLACE TABLE predictions AS SELECT y, predict_duck(X) as y_pred FROM test_data

,Count
0,15


In [91]:
%%dql -co con
-- evaluate model on test sample
SELECT
    COUNT(*) AS total_predictions,
    SUM(CASE WHEN y = y_pred THEN 1 ELSE 0 END) AS correct_predictions,
    CAST(SUM(CASE WHEN y = y_pred THEN 1 ELSE 0 END) AS FLOAT) / COUNT(*) AS accuracy
FROM predictions;

,total_predictions,correct_predictions,accuracy
0,15,13.0,0.866667


In [92]:
con.close()